In [1]:
%load_ext autoreload

%autoreload 2
%env OMP_NUM_THREADS=8

env: OMP_NUM_THREADS=8


In [2]:
import sys, os, glob
sys.path += ['.']

In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from flows.glow import GlowFlow
from flows.invert import Invert
from flows.glow.affine_coupling import coupling_nn_glow
from flows.glow.gaussianize import gaussianize
from models.flvm import FlowLVM

import matplotlib.pyplot as plt
import h5py
from matplotlib.colors import LogNorm, SymLogNorm

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

In [5]:
dset_dir = '/home/cluster/hlasco/bulk1/GAN_dataset/'
fList_t  = glob.glob(dset_dir+'/train/*/processed_data/snapshot.h5')
fList_v  = glob.glob(dset_dir+'/validation/*/processed_data/snapshot.h5')
nFiles_t = len(fList_t)

ps_h = 64
ps_l = ps_h//4

# read .h5 file
def readH5(f, n_channels=4):
    f = f.numpy()
    idx = np.random.randint(0,4)
    idy = np.random.randint(0,4)
    idz = np.random.randint(0,4)
    with h5py.File(f, 'r') as fi:
        u_hr = np.array(fi['HR/ux'][idx*ps_h:(idx+1)*ps_h,idy*ps_h:(idy+1)*ps_h,idz*ps_h:(idz+1)*ps_h], dtype=np.float32).reshape(ps_h,ps_h,ps_h,1)
        u_lr = np.array(fi['LR/ux'][idx*ps_l:(idx+1)*ps_l,idy*ps_l:(idy+1)*ps_l,idz*ps_l:(idz+1)*ps_l], dtype=np.float32).reshape(ps_l,ps_l,ps_l,1)
        v_hr = np.array(fi['HR/uy'][idx*ps_h:(idx+1)*ps_h,idy*ps_h:(idy+1)*ps_h,idz*ps_h:(idz+1)*ps_h], dtype=np.float32).reshape(ps_h,ps_h,ps_h,1)
        v_lr = np.array(fi['LR/uy'][idx*ps_l:(idx+1)*ps_l,idy*ps_l:(idy+1)*ps_l,idz*ps_l:(idz+1)*ps_l], dtype=np.float32).reshape(ps_l,ps_l,ps_l,1)
        w_hr = np.array(fi['HR/uz'][idx*ps_h:(idx+1)*ps_h,idy*ps_h:(idy+1)*ps_h,idz*ps_h:(idz+1)*ps_h], dtype=np.float32).reshape(ps_h,ps_h,ps_h,1)
        w_lr = np.array(fi['LR/uz'][idx*ps_l:(idx+1)*ps_l,idy*ps_l:(idy+1)*ps_l,idz*ps_l:(idz+1)*ps_l], dtype=np.float32).reshape(ps_l,ps_l,ps_l,1)
        r_hr = np.array(fi['HR/rho'][idx*ps_h:(idx+1)*ps_h,idy*ps_h:(idy+1)*ps_h,idz*ps_h:(idz+1)*ps_h], dtype=np.float32).reshape(ps_h,ps_h,ps_h,1)
        r_lr = np.array(fi['LR/rho'][idx*ps_l:(idx+1)*ps_l,idy*ps_l:(idy+1)*ps_l,idz*ps_l:(idz+1)*ps_l], dtype=np.float32).reshape(ps_l,ps_l,ps_l,1)

        m_vel = np.mean([u_lr, v_lr, w_lr])
        s_vel = np.std([u_lr, v_lr, w_lr])

        r_hr = np.log10(r_hr)
        r_lr = np.log10(r_lr)

        m_r = np.mean(r_lr)
        s_r = np.std(r_lr)

        u_hr = (u_hr - np.mean(u_lr))/np.std(u_lr)
        u_lr = (u_lr - np.mean(u_lr))/np.std(u_lr)

        v_hr = (v_hr - np.mean(v_lr))/np.std(v_lr)
        v_lr = (v_lr - np.mean(v_lr))/np.std(v_lr)

        w_hr = (w_hr - np.mean(w_lr))/np.std(w_lr)
        w_lr = (w_lr - np.mean(w_lr))/np.std(w_lr)

        r_hr = (r_hr - m_r)/s_r
        r_lr = (r_lr - m_r)/s_r

        ret_lr = np.concatenate((u_lr, v_lr, w_lr, r_lr), axis=-1)
        if n_channels==4:
            ret_hr = np.concatenate((u_hr, v_hr, w_hr, r_hr), axis=-1)
        elif n_channels==3:
            ret_hr = np.concatenate((u_hr, v_hr, w_hr), axis=-1)
        elif n_channels==1:
            ret_hr = r_hr
        return ret_lr, ret_hr 



def readH5_wrapper(filename, n_channels=4):
    # Assuming your data and labels are float32
    # Your input is parse_function, who arg is filename, and you get X and y as output
    # whose datatypes are indicated by the tuple argument
    lr, hr = tf.py_function(readH5, [filename, n_channels], (tf.float32, tf.float32))
    return hr, lr

batch_size = 1
n_channels = 1
# Create dataset of filenames.
X_train_ds = tf.data.Dataset.from_tensor_slices(fList_t)
X_train_ds = X_train_ds.map(lambda x: readH5_wrapper(x, n_channels=n_channels))
X_train_ds = X_train_ds.batch(batch_size).repeat()

In [6]:
iterator = iter(X_train_ds)

In [7]:
hr, lr = iterator.get_next()
print(lr.shape, hr.shape)

(1, 16, 16, 16, 4) (1, 64, 64, 64, 1)


In [90]:
n_channels=4
dim=3
kwargs_nn={'dim':dim, 'min_filters':8, 'max_filters':128, 'num_blocks':0}

affine_coupling = coupling_nn_glow(**kwargs_nn)
inpt_shape = tf.TensorShape((None,None,None,None,n_channels))

glow = Invert(GlowFlow(
                   dim=dim,
                   upfactor=2,
                   num_layers=3,
                   depth=1,
                   coupling_nn_ctor=affine_coupling))

learning_rate = 2.0e-4
prior = tfp.distributions.Normal(loc=0.0, scale=1.0)
opt_flow = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model_8_128_rho = FlowLVM(glow, prior,
                dim=dim,
                num_bins=16,
                input_channels=n_channels,
                optimizer_flow=opt_flow)



(None, None, None, None, 16)
(None, None, None, 16)
(None, None, None, None, 64)
(None, None, None, 64)
(None, None, None, None, 512)
(None, None, None, 512)


In [91]:
print(model_8_128_rho.param_count())

tf.Tensor(4592768, shape=(), dtype=int32)


In [92]:
model_8_128_rho.test(shape=32)

Testing individual layers:
Testing glow_flow_layer0_squeeze
	 passed
	 Num params: 0
Testing glow_flow_layer0_step0_act_norm
	Error on forward inverse pass:
		x-F^{-1}oF(x): 5.532339e-10
		ildj+fldj: 0.0
	 passed
	 Num params: tf.Tensor(64, shape=(), dtype=int32)
Testing glow_flow_layer0_step0_inv_conv
	Error on forward inverse pass:
		x-F^{-1}oF(x): -8.531895e-10
		ildj+fldj: 0.0
	 passed
	 Num params: tf.Tensor(3136, shape=(), dtype=int32)
Testing glow_flow_layer0_step0_affine_coupling
	Error on forward inverse pass:
		x-F^{-1}oF(x): 0.0
		ildj+fldj: 0.0
	 passed
	 Num params: tf.Tensor(7104, shape=(), dtype=int32)
Testing glow_flow_layer0_split
	 passed
	 Num params: 138400
Testing glow_flow_layer1_squeeze
	 passed
	 Num params: 0
Testing glow_flow_layer1_step0_act_norm
	Error on forward inverse pass:
		x-F^{-1}oF(x): 7.115057e-10
		ildj+fldj: 0.0
	 passed
	 Num params: tf.Tensor(256, shape=(), dtype=int32)
Testing glow_flow_layer1_step0_inv_conv
	Error on forward inverse pass:
		x-

In [93]:
test = model_8_128_rho.sample(n=1, shape=64)

In [94]:
print(test.shape)

(1, 64, 64, 64, 4)


In [82]:
fwd_s([16,None],2)

[8, None]

In [89]:
a=[2,4,6]
for b in range (0,len(a)):
    a.insert(b*2+1,1)
print(a)

[2, 1, 4, 1, 6, 1]
